<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Agente simple</center></span>

<center>Introducción</center>

<figure>
<center>
<img src="../Imagenes/cara-alambre.jpg" width="600" height="600" align="center"/>
</center>
</figure>


Fuente: [Pixabay](https://pixabay.com/images/search/robot/)

##   <span style="color:blue">Autores</span>

- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Richard S. Sutton, Andrew G. Barto, Reinforcement learning: an introduction, 2nd edition, 2020](http://library.lol/main/6502B74CE247C4CD4D4FB54747AD7C7E)
1. [Praveen Palanisamy - Hands-On Intelligent Agents with OpenAI Gym_ Your Guide to Developing AI Agents Using Deep Reinforcement Learning, 2020](http://library.lol/main/E4FD128CF9B93E0F7A542B053330517A)
1. [Turing Paper 1936](http://www.thocp.net/biographies/papers/turing_oncomputablenumbers_1936.pdf)

## <span style="color:blue">Contenido</span>

*  [Introducción](#Introducción)
* [Anatomía de un agente](#Anatomía-de-un-agente)
    * [Clase Environment](#Clase-Environment)
    * [Clase Agent](#Clase-Agent)
    * [Clase Action](#Clase-Action)
    * [Clase Trainer](#Clase-Trainer)
    * [Ejecución de un episodio](#Ejecución-de-un-episodio)

## <span style="color:blue">Introducción</span>

En esta lección exploramos una implementación en Python de un agente en un situación muy simple.

## <span style="color:blue">Anatomía de un agente</span>

El ambiente entregará una recompensa aleatoria al agente durante un número limitado de pasos. El ejemplo es extremadamente simple, pero muestra la interacción entre el agente y el ambiente.

Implementamos las siguientes  clases:

* Environment
* Action
* Agent
* Trainer

Al final se tiene el código para ejecutar un entrenamiento.


### <span style="color:#4CC9F0">Clase Environment</span>

`Enviroment` es la clase encargada del manejo de ambiente. El constructor `__init__()` recibe el número de pasos que recorrerá un agente. El agente se moverá en tres dimensiones en la diagonal definida por los puntos $(x,x,x)$, en donde $x$ será un número entero. Prevemos que el agente dará un paso en el sentido positivo con probabilidad 0.6 y en sentido contrario con probabilidad 0.4. En cada paso el agente recibe una recompensa positiva, si avanza en el sentido positivo y negativa en caso contrario. El método is `is_done()` es implementado para que el ambiente indique que un episodio ha sido completado.

El método `step()` implementa la interfaz (API) para el agente. Es mediante este método que el agente se comunicará con el ambiente.

In [24]:
import numpy as np
from typing import List

class Environment:
    """Define una clase Enviroment básica. 
    se adimitiran 10 recompensas aleatorias"""
    
    def __init__(self, steps=10):
        self.steps = steps
        self.steps_left = steps
        self.current_state = np.array([0.0, 0.0, 0.0])
    
    # regresa una lista de floats (type annotations)
    # usualmente esta es una función del estado actual del Environment
    def next_observation(self, action):
        if action == 1:
            self.current_state += 1.0
        else:
            self.current_state -= 1.0
        
        return self.current_state 
       
    def reward(self,  action):
        r = np.abs(random.random())
        if action == 0:
            r = -r
        return r
        
    # determina si un episodio ha terminado o no
    def is_done(self):
        return self.steps_left == 0
    
    def reset(self, steps=None):
        self.steps = self.steps if steps is None else steps
        self.steps_left = self.steps if steps is None else steps
        self.current_state = np.array([0.0, 0.0, 0.0])   
    
    def step(self, action):
        if self.steps_left == 0:
            raise Exception('El episodio ya fué completado!!')
        self.steps_left -= 1
        observation = self.next_observation(action)
        reward = self.reward(action)
        is_done = self.is_done()
        return observation, reward, is_done, _
    
    def get_steps(self):
        return self.steps
    

### <span style="color:#4CC9F0">Clase Action</span>

Esta clase implementa el generador de acciones. En este ejemplo, cada acción 0 o 1 respectivamente es generada siguiendo la distribución de probabilidad definida por el vector $[p_1, p_2]$.

In [3]:
class Action:
    """
    Clase acción. Define una acción aleatoria
    """
    def __init__(self, probs =[0.4, 0.6], actions=[0, 1]):
        self.probs = probs
        self.actions = actions
    
    def __call__(self):
        a = np.random.choice(len(self.actions), p=self.probs)
        return self.actions[a]

### <span style="color:#4CC9F0">Clase Agent</span>

Esta clase implementa un agente muy simple. El agente recibe una acción generada por su motor de acciones y se la entrega al ambiente mediante la interfaz `step` del ambiente

In [20]:
import random

class Agent:
    """Define una clase Agent básica. 
    lleva la cuenta de la recompensa acumulada"""
    def __init__(self):
        self.total_reward = 0.0
        self.current_observation = [0, 0, 0]
        self.action = None
        self.env = None
    
    def reset(self):
        self.total_reward = 0.0
        self.current_observation = [0, 0, 0]
        
    # acepta la instancia de Environment y hace las siguientes tareas:
    # 1. Observa el ambiente
    # 2. Toma un decisión sobre que acción tomar basado en las observaciones
    # 3. somete la accion al ambiente
    # 4. recibe la recompensa del paso actual y la acumula
    
    def set_config(self, action, env):
        self.action = action
        self.env = env
    
    def take_action(self):
        # selecciona una acción
        action = self.action()
        # informa al ambiente de la acción y recibe retroalimentación
        observation, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward
        self.current_observation = observation
        return observation, reward, is_done, _
    
    def get_total_reward(self):
        return self.total_reward
    
    def get_current_observation(self):
        return self.current_observation
    
   

### <span style="color:#4CC9F0">Clase Trainer</span>

Esta clase entrenará los agentes para logren recompensas. *Trainer* instancia un motor de acciones, un ambiente y un agente- El método `episode()` ejecuta un episodio. El método `reset()` reconfigura el ambiente y el agente. Para el ambiente es posible cambiar el numero de pasos en el episodio.

In [18]:
class Trainer():
    def __init__(self,  agent, action, env):
        """
        Instancia objetos para el entrenamiento
        """
        self.action = action
        self.env = env
        self.agent = agent
        # configura al agente
        self.agent.set_config(action, env)

    def reset(self, steps=None):
        self.env.reset(steps)
        self.agent.reset()
        
    def episode(self):
        is_done = False
        while not is_done:
            observation, reward, is_done, aux_inf = self.agent.take_action()
        print("Episodio completado")
        print("Recompensa total recibida: %.4f" % self.agent.get_total_reward())
        print("Posición final: ", self.agent.get_current_observation())
        print('Número de pasos', env.get_steps())

### <span style="color:#4CC9F0">Ejecuta un episodio de entrenamiento</span>

In [25]:
import numpy as np

# configura el Trainer
steps = 10
action = Action()
env = Environment(steps)
agent = Agent()
trainer = Trainer(agent, action, env)

# corre episodio
trainer.episode()



Episodio completado
Recompensa total recibida: 1.0727
Posición final:  [2. 2. 2.]
Número de pasos 10


In [26]:
trainer.reset(15)
trainer.episode()

Episodio completado
Recompensa total recibida: 1.0015
Posición final:  [3. 3. 3.]
Número de pasos 15


In [16]:
trainer.episode()

Exception: El episodio ya fué completado!!